In [169]:
import pandas as pd
import numpy as np

In [170]:
train = pd.read_csv("../MachineHackODIMatchWinner/Train.csv")
test = pd.read_csv("../MachineHackODIMatchWinner/Test.csv")
sf = pd.read_excel("../MachineHackODIMatchWinner/Sample_submission.xlsx")

#sf = pd.read_excel("../MachinehackGlassQualityPrediction/Glass_Quality_Participants_Data/Sample_Submission.xlsx")


In [171]:
nrow = train.shape[0]
train['isTrain'] =1
test['MatchWinner'] =-1
test['isTrain'] =0

data = pd.concat([train,test],axis=0,sort = False).reset_index(drop=True)

In [172]:
data['Home2Inning'] = 'OH yeah'
for i in data.index:
    if(data['Team1_Venue'][i] == 'Home' and data['Team1_Innings'][i]=='Second'):
            data['Home2Inning'][i] ='Yes'
    else:
            data['Home2Inning'][i] = 'No'
    

/home/ryan/.local/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/home/ryan/.local/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [173]:
data['Neutral2Inning'] = 'foo'

for i in data.index:
    if(data['Team1_Venue'][i] == 'Neutral' and data['Team1_Innings'][i]=='Second'):
            data['Neutral2Inning'][i] = 'Yes'
    else:
            data['Neutral2Inning'][i] = 'No'
    

/home/ryan/.local/lib/python3.6/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  import sys
/home/ryan/.local/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """


In [174]:
data['Away2Inning'] = 'foo'
for i in data.index:
    if(data['Team1_Venue'][i] == 'Home' and data['Team1_Innings'][i]=='Second'):
            data['Away2Inning'][i] ='Yes'
    else:
            data['Away2Inning'][i] ='No'
    

/home/ryan/.local/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.
/home/ryan/.local/lib/python3.6/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [175]:
data =  data.drop(['Team2_Innings','Team2_Venue'], axis=1)

In [176]:
train = data[data['isTrain']==1]
test = data[data['isTrain']==0]


In [177]:
train = train.drop(['isTrain'],axis=1)
test = test.drop(['MatchWinner', 'isTrain'],axis=1)

In [178]:
def onhotencoder(train,test,cols):
    
    from sklearn.preprocessing import OneHotEncoder
    #object_cols = catVar1(train) #catVar1 gives desired categorical column and not all object columns
    object_cols=cols
    print(object_cols)
    
    OH_encoder = OneHotEncoder(handle_unknown='ignore', sparse=False)
    OH_cols_train = pd.DataFrame(OH_encoder.fit_transform(train[object_cols]))
    OH_cols_test = pd.DataFrame(OH_encoder.transform(test[object_cols]))

    # One-hot encoding removed index; put it back
    OH_cols_train.index = train.index
    OH_cols_test.index = test.index

    ##hack for restoring columns names just like get dummies
    column_name = OH_encoder.get_feature_names(object_cols)
    OH_cols_train.columns = column_name
    OH_cols_test.columns = column_name
    

    # Remove desired categorical columns (will replace with one-hot encoding)
    num_train = train.drop(object_cols, axis=1)
    num_test = test.drop(object_cols, axis=1)

    # Add one-hot encoded columns to numerical/remaining features
    OH_train = pd.concat([num_train, OH_cols_train], axis=1)
    OH_test = pd.concat([num_test, OH_cols_test], axis=1)
    
    print(OH_train.shape,OH_test.shape)
    
    
    return OH_train,OH_test

In [179]:
train.columns

Index(['Team1', 'Team2', 'Stadium', 'HostCountry', 'Team1_Venue',
       'Team1_Innings', 'MonthOfMatch', 'MatchWinner', 'Home2Inning',
       'Neutral2Inning', 'Away2Inning'],
      dtype='object')

In [180]:
trainlblohe ,testlblohe = onhotencoder(train,test, cols = [ 'Team1_Venue', 'Team1_Innings','MonthOfMatch','Home2Inning','Neutral2Inning','Away2Inning']) 

['Team1_Venue', 'Team1_Innings', 'MonthOfMatch', 'Home2Inning', 'Neutral2Inning', 'Away2Inning']
(2508, 28) (1075, 27)


In [181]:
y = trainlblohe.MatchWinner

# To keep things simple, we'll use only numerical predictors
X = trainlblohe.drop(['MatchWinner'], axis=1)



In [182]:
from sklearn.calibration import CalibratedClassifierCV
from xgboost import XGBClassifier
from sklearn.model_selection import StratifiedShuffleSplit

In [183]:


clf2 = XGBClassifier(subsample= 1.0, n_estimators= 125, min_child_weight= 1, max_depth= 4,
                         learning_rate=0.2, gamma= 0.5, colsample_bytree= 0.6,random_state=2)  
    

In [184]:
clf2

XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.6, gamma=0.5,
              learning_rate=0.2, max_delta_step=0, max_depth=4,
              min_child_weight=1, missing=None, n_estimators=125, n_jobs=1,
              nthread=None, objective='binary:logistic', random_state=2,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1.0, verbosity=1)

In [185]:
clf2.fit(X,y)


XGBClassifier(base_score=0.5, booster='gbtree', colsample_bylevel=1,
              colsample_bynode=1, colsample_bytree=0.6, gamma=0.5,
              learning_rate=0.2, max_delta_step=0, max_depth=4,
              min_child_weight=1, missing=None, n_estimators=125, n_jobs=1,
              nthread=None, objective='multi:softprob', random_state=2,
              reg_alpha=0, reg_lambda=1, scale_pos_weight=1, seed=None,
              silent=None, subsample=1.0, verbosity=1)

In [186]:
sss = StratifiedShuffleSplit(n_splits=10, test_size=0.4, random_state=3)
sss.get_n_splits(X, y)
    
clf_sigmoid = CalibratedClassifierCV(clf2, cv=sss, method='isotonic')
clf_sigmoid.fit(X, y)
preds = clf_sigmoid.predict_proba(testlblohe)


In [187]:
preds

array([[0.        , 0.        , 0.54530419, ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.55564207, 0.        , ..., 0.        , 0.44435793,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       ...,
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.36697124, 0.        ,
        0.        ],
       [0.        , 0.        , 0.        , ..., 0.        , 0.        ,
        0.        ]])

In [188]:
prediction = pd.DataFrame(data =preds, columns=sf.columns)


In [189]:
prediction

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,0.0,0.000000,0.545304,0.0,0.437899,0.012548,0.000000,0.004249,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
1,0.0,0.555642,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.444358,0.000000
2,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000459,0.000000,0.0,0.357386,0.642155,0.0,0.000000,0.000000,0.000000,0.000000
3,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.405500,0.594500,0.0,0.000000,0.000000,0.000000,0.000000
4,0.0,0.000000,0.000000,0.0,0.000000,0.774574,0.000496,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.224930
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1070,0.0,0.000000,0.000000,0.0,0.000000,0.837566,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.162434
1071,0.0,0.541563,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.458437,0.000000,0.000000,0.000000
1072,0.0,0.000000,0.000000,0.0,0.000000,0.568399,0.000000,0.000000,0.0,0.000000,0.431601,0.0,0.000000,0.000000,0.000000,0.000000
1073,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.0,0.000000,0.633029,0.0,0.000000,0.366971,0.000000,0.000000


In [190]:
#manipulate prediction

In [191]:
man_pred = prediction.copy()

In [192]:
for row in range(len(testlblohe)):
    team1,team2 = testlblohe.iloc[row].Team1,testlblohe.iloc[row].Team2
    d = list({col for col in prediction.columns} - {team1,team2})
    for i in d:
        man_pred.iloc[row][i] =0.0
               
        

In [193]:
man_pred

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15
0,0.0,0.000000,0.545304,0.0,0.437899,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.000000
1,0.0,0.555642,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.444358,0.000000
2,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.357386,0.642155,0.0,0.000000,0.000000,0.000000,0.000000
3,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.405500,0.594500,0.0,0.000000,0.000000,0.000000,0.000000
4,0.0,0.000000,0.000000,0.0,0.000000,0.774574,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.224930
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1070,0.0,0.000000,0.000000,0.0,0.000000,0.837566,0.0,0.0,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.162434
1071,0.0,0.541563,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.000000,0.0,0.458437,0.000000,0.000000,0.000000
1072,0.0,0.000000,0.000000,0.0,0.000000,0.568399,0.0,0.0,0.0,0.000000,0.431601,0.0,0.000000,0.000000,0.000000,0.000000
1073,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.0,0.0,0.000000,0.633029,0.0,0.000000,0.366971,0.000000,0.000000


In [194]:
man_pred.to_excel('../MachineHackODIMatchWinner/calxgbtunedmanpred.xlsx')
